In [7]:
'''
Chat G api version

'''

'\nChat G api version\n\n'

In [8]:
import os
from glob import glob

In [9]:
root_path = "/disk3/eric/VG_arma3_roboflow"
root_path_img = root_path + "/images/*.jpg"

In [10]:
img_paths = sorted(glob(root_path_img))

In [11]:
import base64
import requests

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')


#----------------------------------------------
def caption_with_chatg(txt_prompt, image_path,api_key):

    # Getting the base64 string
    base64_image = encode_image(image_path)

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    payload = {
        "model": "gpt-4o",
        "messages": [
        {
            "role": "user",
            "content": [
            {
                "type": "text",
                "text": f"{txt_prompt}"
            },
            {
                "type": "image_url",
                "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
                }
            }
            ]
        }
        ],
        "max_tokens": 300
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload).json()
    #
    z = response['choices'][0]['message']['content']
    print("content :",z)
    return z 

In [15]:
import collections
tmp_ = collections.defaultdict()

# option 1 : Describe the image implicitly refer to the ontology below. => 암묵적 허용을 해줘서 서술. 
# option 2 : Describe the image based on the ontology below. => 명시적으로 온톨로지들을 기준으로 정리. 


text_prompt = '''

Describe the image based on the ontology below.

entity: soldier, kneeling soldier, dead soldier, checkpoint, burning vehicle, ground, road, gun, tank

'''



#--- single try
image_path = "/disk3/eric/VG_arma3_roboflow/images/frame_2800.jpg"

caption_r = caption_with_chatg(text_prompt, image_path, api_key)


content : The image features the following entities:

- Kneeling soldier: There is a soldier kneeling near foliage.
- Soldier: Another soldier is visible on the road.
- Gun: A gun is visible in the foreground of the image.
- Road: A road runs through the scene, flanked by trees and vegetation.


In [13]:
#--- multiple images

for path in img_paths:
    try:
        print(path)
        img_id = path.split("/")[-1]
        tmp_[img_id] = caption_with_chatg(text_prompt, path, api_key)
        
    except Exception as e:
        print(f"error : {e}")
        pass


/disk3/eric/VG_arma3_roboflow/images/frame_2390.jpg
content : The image depicts a road surrounded by dry grass and trees. There is visible dust, possibly indicating recent activity or movement along the road. No soldiers, vehicles, or other entities from the provided ontology are observed.
/disk3/eric/VG_arma3_roboflow/images/frame_2400.jpg
content : ground, road
/disk3/eric/VG_arma3_roboflow/images/frame_2410.jpg
content : The image depicts a road and surrounding ground area. There is a clear sky and trees lining the road. A sign is visible on the side of the road. No soldiers, burning vehicles, checkpoints, guns, or tanks are observed.
/disk3/eric/VG_arma3_roboflow/images/frame_2420.jpg
content : The image shows a road surrounded by trees and vegetation. There is a visible road sign and tracks on the ground. There are no observable entities like soldiers, checkpoints, burning vehicles, guns, or tanks.
/disk3/eric/VG_arma3_roboflow/images/frame_2430.jpg
content : The image depicts a r

KeyboardInterrupt: 

In [37]:
# Get current time
from datetime import datetime, timedelta


current_time = datetime.now()

print("Current Time:", current_time.strftime("%Y-%m-%d %H:%M"))

cnt = 0
all_txts = ""

for k,v in tmp_.items():
    cnt +=1
    current_time += timedelta(minutes=3)
    current_time_v = current_time.strftime("%Y-%m-%d %H:%M")
    print(current_time_v)
    print(v)
    all_txts += current_time_v + ":" 
    all_txts += v

Current Time: 2024-12-12 14:23
2024-12-12 14:26
The image shows a road with visible tire tracks on the ground. There are trees and vegetation on either side of the road. An object, possibly a tank, is in the distance. The scene appears to be outdoors with a clear blue sky. There are no visible soldiers, burning vehicles, or checkpoints.
2024-12-12 14:29
The image depicts a road surrounded by vegetation and trees. It's a clear, sunny day, with no visible entities such as soldiers, checkpoints, burning vehicles, guns, or tanks. The ground is dry and dusty.
2024-12-12 14:32
The image shows a road leading into the distance with some dust or smoke. There are trees on both sides of the road. There is a triangular road sign visible on the right, but no specific entities like soldiers, vehicles, or checkpoints are present in the image.
2024-12-12 14:35
The image contains a road and ground. There are no visible soldiers, kneeling soldiers, dead soldiers, checkpoints, burning vehicles, guns, or 

In [38]:
all_txts

"2024-12-12 14:26:The image shows a road with visible tire tracks on the ground. There are trees and vegetation on either side of the road. An object, possibly a tank, is in the distance. The scene appears to be outdoors with a clear blue sky. There are no visible soldiers, burning vehicles, or checkpoints.2024-12-12 14:29:The image depicts a road surrounded by vegetation and trees. It's a clear, sunny day, with no visible entities such as soldiers, checkpoints, burning vehicles, guns, or tanks. The ground is dry and dusty.2024-12-12 14:32:The image shows a road leading into the distance with some dust or smoke. There are trees on both sides of the road. There is a triangular road sign visible on the right, but no specific entities like soldiers, vehicles, or checkpoints are present in the image.2024-12-12 14:35:The image contains a road and ground. There are no visible soldiers, kneeling soldiers, dead soldiers, checkpoints, burning vehicles, guns, or tanks.2024-12-12 14:38:The image 

In [40]:
tmp_["all_txts"] = all_txts

In [41]:
# Save to JSON file
import json
with open('/home/eric/srcs/Graph_Lab/sample_data/all_output_1220.json', 'w') as json_file:
    json.dump(tmp_, json_file)

In [42]:
tmp_

defaultdict(None,
            {'frame_2390.jpg': 'The image shows a road with visible tire tracks on the ground. There are trees and vegetation on either side of the road. An object, possibly a tank, is in the distance. The scene appears to be outdoors with a clear blue sky. There are no visible soldiers, burning vehicles, or checkpoints.',
             'frame_2400.jpg': "The image depicts a road surrounded by vegetation and trees. It's a clear, sunny day, with no visible entities such as soldiers, checkpoints, burning vehicles, guns, or tanks. The ground is dry and dusty.",
             'frame_2410.jpg': 'The image shows a road leading into the distance with some dust or smoke. There are trees on both sides of the road. There is a triangular road sign visible on the right, but no specific entities like soldiers, vehicles, or checkpoints are present in the image.',
             'frame_2420.jpg': 'The image contains a road and ground. There are no visible soldiers, kneeling soldiers, de